In [1]:
CLIENT_SECRETS_FILE = "client_secret.json"

In [2]:
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

In [3]:
import google.oauth2.credentials
import os
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
 
def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [4]:
if __name__ == '__main__':
    # When running locally, disable OAuthlib's HTTPs verification. When
    # running in production *do not* leave this option enabled.
    os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
    service = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=165489355719-g7manvp37g4s4h2s8t59sm1hn5l6sdjv.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=o0QFCwZ8VBKZjdlRScddpJ4rxk5G8M&prompt=consent&access_type=offline
Enter the authorization code: 4/5QFvSQpzhdZ2OLu_pK-3eZHPKw8zQRx_4uMhIVy91KoHMcR8Y9TseKk


In [5]:
import pickle
from google.auth.transport.requests import Request

...
...

def get_authenticated_service():
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    #  Check if the credentials are invalid or do not exist
    if not credentials or not credentials.valid:
        # Check if the credentials have expired
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_console()

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)

    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [11]:
def get_videos(service, **kwargs):
    final_results = []
    results = service.search().list(**kwargs).execute()

    i = 0
    max_pages = 3
    while results and i < max_pages:
        final_results.extend(results['items'])

        # Check if another page exists
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.search().list(**kwargs).execute()
            i += 1
        else:
            break

    return final_results

def search_videos_by_keyword(service, **kwargs):
    results = get_videos(service, **kwargs)
    for item in results:
        print('%s - %s' % (item['snippet']['title'], item['id']['videoId']))


In [8]:
def get_video_comments(service, **kwargs):
    comments = []
    results = service.commentThreads().list(**kwargs).execute()

    while results:
        for item in results['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)

        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.commentThreads().list(**kwargs).execute()
        else:
            break

    return comments


In [12]:
keyword = input('Enter a keyword: ')
search_videos_by_keyword(service, q=keyword, part='id,snippet', eventType='completed', type='video')

Digital nomad: Digital nomad
4 Best Places in The World For Digital Nomads - wGmUzsZ37B8
Digital Nomad Travel Podcast with ChristabellaTravels from Lisbon, Portugal - vXLD8Gw6XS8
Pros &amp; Cons of a Digital Nomad Lifestyle 🌴[Podcast Ep22] - EoRdSTEf0Sw
Living in Estonia for Digital Nomads - R88m2juxCcw
Digital Nomad Taxes - 5 Biggest Misconceptions - Location Independent Lifestyle - 6X4oRhz1Yhs
Dating, Love, and Relationships as a Digital Nomad - Location Independent Live Stream - Uehenx13ydE
How This Traditional Accountant Became a Digital Nomad CPA! [Owner of NomadTax.io] - iVBH0a_ugxE
Four Ways to Become a Digital Nomad (Live Stream) - kBkvOto3qd0
How to Meet People and Make Friends as a Digital Nomad - GU7no98tpoQ
Digital Nomad Apartment Tour in Miami | Nomad Queen - -TysAqQEbeg
Finding Non-Tech Jobs as a Digital Nomad - Remote Work Location Independent Lifestyle - DFJldUbHwsE
What is a Digital Nomad, Anyway? - 3WStibGGar0
Digital Nomads - How To Become A Full Time Digital Nomad I